In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [3]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
import os

caffe_root = os.getcwd() + '../caffe_priv'  # this file should be run from {caffe_root}/examples (otherwise change this line)
proj_root = os.getcwd() + '/../'
sys.path.insert(0, caffe_root + 'python')
sys.path.insert(0, proj_root + 'lib')
import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

In [4]:

if os.path.isfile(proj_root + 'output/faster_rcnn_end2end/trainval/save/zf8_faster_rcnn_iter_160000.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Downloading pre-trained CaffeNet model...'

CaffeNet found.


In [1]:
import _init_paths
from fast_rcnn.config import cfg 
from fast_rcnn.test import im_detect                                                                                                                                       
from fast_rcnn.nms_wrapper import nms 
from utils.timer import Timer                                                                                                                                              
import matplotlib.pyplot as plt 
import numpy as np
import scipy.io as sio 
import cv2 
import argparse

def vis_detections(im, class_name, dets, thresh=0.5):
    """Draw detected bounding boxes."""
    inds = np.where(dets[:, -1] >= thresh)[0]
    if len(inds) == 0:
        return
 
    im = im[:, :, (2, 1, 0)]
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')
    for i in inds:
        bbox = dets[i, :4]
        score = dets[i, -1]
 
        ax.add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5)
            )
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')                                                                                                                                
 
    ax.set_title(('{} detections with '
                  'p({} | box) >= {:.1f}').format(class_name, class_name,
                                                  thresh),
                  fontsize=14)
    plt.axis('off')
    plt.tight_layout()

def detection(net, image_name):
    """Detect object classes in an image using pre-computed object proposals."""
               
    # Load the demo image
    im_file = os.path.join(cfg.DATA_DIR, 'demo', image_name)
    im = cv2.imread(im_file)
               
    # Detect all object classes and regress object bounds
    timer = Timer()
    timer.tic()
    scores, boxes = im_detect(net, im)
    timer.toc()
    print ('Detection took {:.3f}s for '
           '{:d} object proposals').format(timer.total_time, boxes.shape[0])
               
    # Visualize detections for each class
    CONF_THRESH = 0.8
    NMS_THRESH = 0.3
    for cls_ind, cls in enumerate(CLASSES[1:]):
        cls_ind += 1 # because we skipped background
        cls_boxes = boxes[:, 4*cls_ind:4*(cls_ind + 1)]
        cls_scores = scores[:, cls_ind]
        dets = np.hstack((cls_boxes,                                                                                                                                       
                          cls_scores[:, np.newaxis])).astype(np.float32)
        keep = nms(dets, NMS_THRESH)
        dets = dets[keep, :]
        vis_detections(im, cls, dets, thresh=CONF_THRESH)



In [ ]:
gpu_id = 2
caffe.set_mode_gpu()
caffe.set_device(gpu_id)
model_def = proj_root + 'models/viva/ZF8/test.prototxt'
model_weights = proj_root + 'output/faster_rcnn_end2end/trainval/save/zf8_faster_rcnn_iter_160000.caffemodel'
cfg.TEST.HAS_RPN = True
cfg.GPU_ID = gpu_id

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

im = 128 * np.ones((960, 1280, 3), dtype=np.uint8)